In [5]:
from typing import Dict, Tuple, List, Any
from pathlib import Path
import pandas as pd
from backtesting import Strategy, Backtest
from kedro.framework.session import KedroSession
from kedro.framework.startup import bootstrap_project

In [2]:
metadata = bootstrap_project(Path.cwd().parent.parent)
with KedroSession.create(metadata.package_name,
        project_path=metadata.project_path,
        env="base",
    ) as session: 
    context = session.load_context()

/usr/local/lib/python3.8/site-packages/kedro/config/config.py:296: UserWarning: Duplicate environment detected! Skipping re-loading from configuration path: /backtest-example/backtest-example/conf/base
  warn(


2022-01-01 11:51:19,164 - kedro.framework.session.store - INFO - `save()` not implemented for `BaseSessionStore`. Skipping the step.


In [22]:
class LgbmModel(Strategy):
    """
    ref: https://kernc.github.io/backtesting.py/doc/backtesting/backtesting.html#backtesting.backtesting.Order&gsc.tab=0
    """

    def init(self):
        pass

    def next(self):
        """
        next() function is executed per bar.
        We can get latest value access to index of [-1].
        """
        # exit
        if self.data.buy_cost[-1]:
            self.sell()
            # self.position.close()

        if self.data.sell_cost[-1]:
            self.buy()
            # self.position.close()

        # entry
        if self.data.buy_entry[-1] and self.data.buy_cost[-1]:
            self.buy()

        if self.data.sell_entry[-1] and self.data.sell_cost[-1]:
            self.sell()

In [15]:
def supertrend_test(df: pd.DataFrame):
    """
    ref: https://gist.github.com/shinseitaro/be1d1ea012b1dc2657569632eb1e9069#file-crypto-advent-calender-2021-ipynb
    """
    bt = Backtest(df, LgbmModel, trade_on_close=True, cash=100000, commission=0.001)
    stats = bt.run()

    return (bt, stats)

In [23]:
predicted = context.catalog.load("include_y_hat")
predicted["buy_entry"] = predicted["y_hat_buy"].values > 0
predicted["sell_entry"] = predicted["y_hat_sell"].values > 0
bt, stats = supertrend_test(predicted)

2022-01-02 01:35:28,534 - kedro.io.data_catalog - INFO - Loading data from `include_y_hat` (PickleDataSet)...


In [24]:
stats

Start                     2021-12-26 14:00:00
End                       2021-12-31 11:10:00
Duration                      4 days 21:10:00
Exposure Time [%]                   99.786629
Equity Final [$]                   103332.016
Equity Peak [$]                     107532.22
Return [%]                           3.332016
Buy & Hold Return [%]               -3.678327
Return (Ann.) [%]                  805.786541
Volatility (Ann.) [%]              516.675983
Sharpe Ratio                         1.559559
Sortino Ratio                       48.858773
Calmar Ratio                       176.544449
Max. Drawdown [%]                   -4.564213
Avg. Drawdown [%]                   -1.040091
Max. Drawdown Duration        1 days 11:55:00
Avg. Drawdown Duration        0 days 06:26:00
# Trades                                    1
Win Rate [%]                            100.0
Best Trade [%]                       3.487372
Worst Trade [%]                      3.487372
Avg. Trade [%]                    

In [25]:
bt.plot()

Row(id='2788', ...)